## Imports

In [1]:
import datetime
import random
import time

import numpy as np
import pandas as pd
import requests
import copy
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

## Helpers

In [2]:
class SeleniumDriver:
    def __init__(self, driver_path=r'chromedriver.exe'):
        options = webdriver.ChromeOptions()
        options.add_argument(r"--user-data-dir=C:\Users\dm1tr\AppData\Local\Google\Chrome\User Data")
        options.add_argument(r'--profile-directory=Default')
        s = Service(driver_path)
        self.driver = webdriver.Chrome(service=s, options=options)

    def get_html(self, url):
        time.sleep(random.randint(1, 3))
        self.driver.get(url)
        time.sleep(random.randint(5, 10))
        return self.driver.page_source

## Parser

In [3]:
class AutoRuParser:
    def __init__(self, driver, result=None):
        self.initial_url = 'https://auto.ru/moskva/cars/bmw/all/'
        self.driver = driver
        if result is None:
            self.__result = {
                'title': [],
                'engine': [],
                'gearbox': [],
                'body': [],
                'wheel': [],
                'condition': [],
                'creation_date': [],
                'views_cnt': [],
                'owners': [],
                'pts': [],
                'accident': [],
                'customs': [],
                'transmission': [],
                'color': [],
                'mileage': [],
                'year': [],
                'real_owner': [],
                'price': [],
                'dealer': [],
            }
        else:
            self.__result = result

        self.__errors = []

    @property
    def result(self):
        return copy.deepcopy(self.__result)

    @property
    def shaved_result(self):
        res_copy = self.result
        minn = np.min(list(map(len, res_copy.values())))
        maxx = np.max(list(map(len, res_copy.values())))

        print(f'max len is {maxx}. shaving all to min = {minn}')
        for k in res_copy.keys():
            res_copy[k] = res_copy[k][:minn]

        return res_copy


    def parse(self):
        page = 1
        while True:
            cur_url = f'{self.initial_url}?page={page}'
            html = self.driver.get_html(cur_url)
            soup = BeautifulSoup(html)
            main_cards = soup.find_all('div', class_='ListingItem__main')
            print(page)

            if soup.find('a', class_='Button_checked').text == '1' and page != 1:
                break

            for main_card in main_cards:
                try:
                    link = main_card.find('a', class_='ListingItemTitle__link')
                    html = self.driver.get_html(link['href'])
                    soup = BeautifulSoup(html)

                    title = soup.find('h1', class_='CardHead__title')
                    if title is None:
                        continue

                    body = soup.find('li', class_='CardInfoRow CardInfoRow_bodytype')
                    if body is None:
                        continue

                    self.__result['title'].append(title.text)
                    benefits = soup.find_all('div', class_='CardBenefits__item-title')

                    real_owner = 0
                    accident = 1
                    for benefit in benefits:
                        if 'Продаёт собственник' in benefit.text:
                            real_owner = 1
                        if 'ДТП не' in benefit.text:
                            accident = 0

                    self.__result['dealer'].append(
                        1 if soup.find('div', class_='CardSellerNamePlace__owner-info_dealer') is not None else 0)

                    self.__result['accident'].append(accident)
                    self.__result['real_owner'].append(real_owner)

                    views_cnt = soup.find('div', class_='CardHead__infoItem CardHead__views')
                    self.__result['views_cnt'].append(views_cnt.text if views_cnt is not None else 0)

                    creation_date = soup.find('div', class_='CardHead__infoItem CardHead__creationDate')
                    self.__result['creation_date'].append(
                        creation_date.text if creation_date is not None else datetime.date.today())

                    year = soup.find('li', class_='CardInfoRow CardInfoRow_year')
                    self.__result['year'].append(year.a.text if year is not None else '')

                    mileage = soup.find('li', class_='CardInfoRow CardInfoRow_kmAge')
                    self.__result['mileage'].append(
                        mileage.find_all('span')[1].text.replace('&nbsp;', '') if mileage is not None else np.nan)

                    body = soup.find('li', class_='CardInfoRow CardInfoRow_bodytype').a.text
                    self.__result['body'].append(body)

                    color = soup.find('li', class_='CardInfoRow CardInfoRow_color').a.text
                    self.__result['color'].append(color)

                    engine = soup.find('li', class_='CardInfoRow CardInfoRow_engine').div.text
                    self.__result['engine'].append(engine)

                    gearbox = soup.find('li', class_='CardInfoRow CardInfoRow_transmission').find_all('span')[1].text
                    self.__result['gearbox'].append(gearbox)

                    transmission = soup.find('li', class_='CardInfoRow CardInfoRow_drive').find_all('span')[1].text
                    self.__result['transmission'].append(transmission)

                    wheel = soup.find('li', class_='CardInfoRow CardInfoRow_wheel').find_all('span')[1].text
                    self.__result['wheel'].append(wheel)

                    condition = soup.find('li', class_='CardInfoRow CardInfoRow_state').find_all('span')[1].text
                    self.__result['condition'].append(condition)

                    owners = soup.find('li', class_='CardInfoRow CardInfoRow_ownersCount').find_all('span')[1].text
                    self.__result['owners'].append(owners)

                    pts = soup.find('li', class_='CardInfoRow CardInfoRow_pts').find_all('span')[1].text
                    self.__result['pts'].append(pts)

                    customs = soup.find('li', class_='CardInfoRow CardInfoRow_customs').find_all('span')[1].text
                    self.__result['customs'].append(customs)

                    price = soup.find('span', class_='OfferPriceCaption__price').text
                    self.__result['price'].append(price)
                except Exception as ex:
                    self.__errors.append((ex, link))
                    print(ex)
                    print('------')
                    print(link)
            page += 1

## Parsing

In [4]:
driver = SeleniumDriver()
parser = AutoRuParser(driver)
parser.parse()

1
2
3
4
5
6
7
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=7
8
9
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=9
10
11
12
13
14
15
16
17
18
19
20
21
22
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=22
23
24
25
26
27
28
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=28
29
30
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=30
31
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=31
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=31
32
33
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=33
34
35
36
'NoneType' object has no attribute 'text'
------
https://auto.ru/moskva/cars/bmw/all/?page=36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
'NoneType' obje

In [13]:
res = parser.shaved_result
df = pd.DataFrame(res)
df

,title,engine,gearbox,body,wheel,condition,creation_date,views_cnt,owners,pts,accident,customs,transmission,color,mileage,year,real_owner,price,dealer
0,BMW 5 серии 520d V (E60/E61) Рестайлинг,2.0 л / 163 л.с. / Дизель,автоматическая,седан,Левый,Не требует ремонта,5 ноября,143 (47 сегодня),2 владельца,Оригинал,0,Растаможен,задний,серый,216 100 км,2007,0,985 000 ₽,0
1,BMW 3 серии 320i VI (F3x),2.0 л / 184 л.с. / Бензин,автоматическая,седан,Левый,Не требует ремонта,6 ноября,51 (51 сегодня),2 владельца,Оригинал,0,Растаможен,задний,серый,72 650 км,2014,0,1 675 000 ₽,0
2,BMW 5 серии 523i VI (F10/F11/F07),2.5 л / 204 л.с. / Бензин,автоматическая,седан,Левый,Не требует ремонта,28 октября,3901 (130 сегодня),3 или более,Оригинал,0,Растаможен,задний,синий,202 000 км,2011,0,1 155 000 ₽,1
3,BMW X5 50i II (E70) Рестайлинг,4.4 л / 407 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,27 октября,7785 (231 сегодня),3 или более,Оригинал,1,Растаможен,полный,чёрный,145 000 км,2012,0,1 499 000 ₽,1
4,BMW X5 3.0i I (E53) Рестайлинг,3.0 л / 231 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,1 ноября,1733 (151 сегодня),3 или более,Оригинал,0,Растаможен,полный,чёрный,219 000 км,2005,0,799 000 ₽,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,BMW X6 30d II (F16),3.0 л / 249 л.с. / Дизель,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,1 октября,809 (2 сегодня),1 владелец,Оригинал,0,Растаможен,полный,чёрный,52 000 км,2019,0,4 800 000 ₽,0
3405,BMW 6 серии 650i II (E63/E64),3.0 л / 249 л.с. / Дизель,автоматическая,купе,Левый,Не требует ремонта,28 октября,461 (2 сегодня),3 или более,Оригинал,1,Растаможен,полный,чёрный,185 500 км,2007,0,1 750 000 ₽,0
3406,BMW X5 30d IV (G05),2.0 л / 190 л.с. / Дизель,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,27 октября,263 (1 сегодня),1 владелец,Оригинал,1,Растаможен,задний,чёрный,60 000 км,2020,1,10 700 000 ₽,0
3407,BMW X5 M50d IV (G05),4.4 л / 600 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,13 сентября,995 (2 сегодня),3 или более,Дубликат,0,Растаможен,полный,чёрный,72 000 км,2019,0,3 350 000 ₽,0


In [14]:
df.to_csv('autos.csv', index=False)

In [6]:
res = parser.shaved_result

In [15]:
df = pd.read_csv('autos.csv')

In [16]:
df

,title,engine,gearbox,body,wheel,condition,creation_date,views_cnt,owners,pts,accident,customs,transmission,color,mileage,year,real_owner,price,dealer
0,BMW 5 серии 520d V (E60/E61) Рестайлинг,2.0 л / 163 л.с. / Дизель,автоматическая,седан,Левый,Не требует ремонта,5 ноября,143 (47 сегодня),2 владельца,Оригинал,0,Растаможен,задний,серый,216 100 км,2007,0,985 000 ₽,0
1,BMW 3 серии 320i VI (F3x),2.0 л / 184 л.с. / Бензин,автоматическая,седан,Левый,Не требует ремонта,6 ноября,51 (51 сегодня),2 владельца,Оригинал,0,Растаможен,задний,серый,72 650 км,2014,0,1 675 000 ₽,0
2,BMW 5 серии 523i VI (F10/F11/F07),2.5 л / 204 л.с. / Бензин,автоматическая,седан,Левый,Не требует ремонта,28 октября,3901 (130 сегодня),3 или более,Оригинал,0,Растаможен,задний,синий,202 000 км,2011,0,1 155 000 ₽,1
3,BMW X5 50i II (E70) Рестайлинг,4.4 л / 407 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,27 октября,7785 (231 сегодня),3 или более,Оригинал,1,Растаможен,полный,чёрный,145 000 км,2012,0,1 499 000 ₽,1
4,BMW X5 3.0i I (E53) Рестайлинг,3.0 л / 231 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,1 ноября,1733 (151 сегодня),3 или более,Оригинал,0,Растаможен,полный,чёрный,219 000 км,2005,0,799 000 ₽,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,BMW X6 30d II (F16),3.0 л / 249 л.с. / Дизель,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,1 октября,809 (2 сегодня),1 владелец,Оригинал,0,Растаможен,полный,чёрный,52 000 км,2019,0,4 800 000 ₽,0
3405,BMW 6 серии 650i II (E63/E64),3.0 л / 249 л.с. / Дизель,автоматическая,купе,Левый,Не требует ремонта,28 октября,461 (2 сегодня),3 или более,Оригинал,1,Растаможен,полный,чёрный,185 500 км,2007,0,1 750 000 ₽,0
3406,BMW X5 30d IV (G05),2.0 л / 190 л.с. / Дизель,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,27 октября,263 (1 сегодня),1 владелец,Оригинал,1,Растаможен,задний,чёрный,60 000 км,2020,1,10 700 000 ₽,0
3407,BMW X5 M50d IV (G05),4.4 л / 600 л.с. / Бензин,автоматическая,внедорожник 5 дв.,Левый,Не требует ремонта,13 сентября,995 (2 сегодня),3 или более,Дубликат,0,Растаможен,полный,чёрный,72 000 км,2019,0,3 350 000 ₽,0
